# Pandas -- Series and DataFrames

Main source: https://github.com/jakevdp/PythonDataScienceHandbook/tree/master/notebooks

**Pandas is a library for fast and efficient computation on big datasets. As in Numpy, many operations in Pandas are vectorized and thus efficient and fast.**

Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a DataFrame. DataFrames are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks (-> relational algebra) and spreadsheet programs.

As we saw, NumPy's ndarray data structure provides essential features for the type of clean, well-organized data typically seen in numerical computing tasks. While it serves this purpose very well, its limitations become clear when we need more flexibility (e.g., attaching labels to data, working with missing data, etc.) and when attempting operations that do not map well to element-wise broadcasting (e.g., groupings, pivots, etc.), each of which is an important piece of analyzing the less structured data available in many forms in the world around us. Pandas, and in particular its Series and DataFrame objects, builds on the NumPy array structure and provides efficient access to these sorts of "data munging" tasks that occupy much of a data scientist's time.

In [1]:
# Just as we import numpy usually as np, we import pandas under the alias of pd. 
# We'll import numpy as well, because we'll need it often when using pandas
import numpy as np
import pandas as pd

## The Pandas Series Object 

A Pandas Series is a one-dimensional array of indexed data. It can be created from a list or array as follows:

In [2]:
data = pd.Series([0.25, 0.5, np.NaN, 1.0])
data

0    0.25
1    0.50
2     NaN
3    1.00
dtype: float64

In [3]:
type(data)

pandas.core.series.Series

In [4]:
data.values, type(data.values)

(array([0.25, 0.5 ,  nan, 1.  ]), numpy.ndarray)

In [5]:
data.index, type(data.index), list(data.index)

(RangeIndex(start=0, stop=4, step=1),
 pandas.core.indexes.range.RangeIndex,
 [0, 1, 2, 3])

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [6]:
data[1:3]

1    0.5
2    NaN
dtype: float64

In [7]:
type(data[1])

numpy.float64

In [8]:
print(dir(data))

['T', '_AXIS_ALIASES', '_AXIS_IALIASES', '_AXIS_LEN', '_AXIS_NAMES', '_AXIS_NUMBERS', '_AXIS_ORDERS', '_AXIS_REVERSED', '_AXIS_SLICEMAP', '__abs__', '__add__', '__and__', '__array__', '__array_prepare__', '__array_priority__', '__array_wrap__', '__bool__', '__bytes__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__rdivmod_

### Series as generalized NumPy array 

From what we've seen so far, it may look like the Series object is basically interchangeable with a one-dimensional NumPy array. The essential difference is the presence of the index: while the Numpy Array has an implicitly defined integer index used to access the values, the Pandas Series has an explicitly defined index associated with the values.

This explicit index definition gives the Series object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type. For example, if we wish, we can use strings as an index:

In [9]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'd', 'c'])
data

a    0.25
b    0.50
d    0.75
c    1.00
dtype: float64

In [10]:
data.index = list("AbCD")
data

A    0.25
b    0.50
C    0.75
D    1.00
dtype: float64

In [11]:
data['b']

0.5

In [12]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[3, 7, 3, 4])
data

3    0.25
7    0.50
3    0.75
4    1.00
dtype: float64

In [13]:
data[3]

3    0.25
3    0.75
dtype: float64

In [14]:
type(data[3])

pandas.core.series.Series

###  Series as specialized dictionary

In this way, you can think of a Pandas Series a bit like a specialization of a Python dictionary. A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a Series is a structure which maps typed keys to a set of typed values. This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas Series makes it much more efficient than Python dictionaries for certain operations.

The Series-as-dictionary analogy can be made even more clear by constructing a Series object directly from a Python dictionary:

In [15]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [16]:
population['Texas']

26448193

Unlike a dictionary, though, the Series also supports array-style operations such as slicing:

In [17]:
population['California':'Illinois'] 
# note that Illionis is included

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

### Constructing Series objects 

In [18]:
# data can be a scalar
pd.Series(5, index=[100, 200, 300])

100    5
200    5
300    5
dtype: int64

In [19]:
# data can be a dictionary
ser = pd.Series({2: 'a', 1: 'b', 3: 'c'})
ser

2    a
1    b
3    c
dtype: object

In [20]:
ser.to_dict()

{2: 'a', 1: 'b', 3: 'c'}

## The Pandas DataFrame Object 

The next fundamental structure in Pandas is the DataFrame. Like the Series object, it can be thought of either as a generalization of a NumPy array, or as a specialization of a Python dictionary. We'll now take a look at each of these perspectives.

### DataFrame as a generalized NumPy array 

If a Series is an analog of a one-dimensional array with flexible indices, a DataFrame is an analog of a two-dimensional array with both flexible row indices and flexible column names. Just as you might think of a two-dimensional array as an ordered sequence of aligned one-dimensional columns, you can think of a DataFrame as a sequence of aligned Series objects. Here, by "aligned" we mean that they share the same index.

To demonstrate this, let's first construct a new Series listing the area of each of the five states discussed in the previous section:

In [21]:
area_dict ={'California': 423967, 'Texas': 695662, 'New York': 141297,
            'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)
area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
dtype: int64

Now that we have this along with the population Series from before, we can use a dictionary to construct a single two-dimensional object containing this information:

In [22]:
states = pd.DataFrame({'population': population,
                       'area': area,
                       'country': 'USA'})
print(states.dtypes)
states

population     int64
area           int64
country       object
dtype: object


,population,area,country
California,38332521,423967,USA
Texas,26448193,695662,USA
New York,19651127,141297,USA
Florida,19552860,170312,USA
Illinois,12882135,149995,USA


This looks like a generalized dictionary! The keys are the names of the state, and the values are like a list [area, gountry, population]

In [23]:
states.sort_values(by="population", ascending=False)

,population,area,country
California,38332521,423967,USA
Texas,26448193,695662,USA
New York,19651127,141297,USA
Florida,19552860,170312,USA
Illinois,12882135,149995,USA


In [24]:
states['population'], type(states['population'])

(California    38332521
 Texas         26448193
 New York      19651127
 Florida       19552860
 Illinois      12882135
 Name: population, dtype: int64, pandas.core.series.Series)

In [25]:
states['population'].idxmax()

'California'

In [26]:
states.loc[states['population'].idxmax()]

population    38332521
area            423967
country            USA
Name: California, dtype: object

In [27]:
states.index

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')

In [28]:
states.columns

Index(['population', 'area', 'country'], dtype='object')

In [29]:
states.values

array([[38332521, 423967, 'USA'],
       [26448193, 695662, 'USA'],
       [19651127, 141297, 'USA'],
       [19552860, 170312, 'USA'],
       [12882135, 149995, 'USA']], dtype=object)

In [30]:
states.loc['California']

population    38332521
area            423967
country            USA
Name: California, dtype: object

In [31]:
type(states.values)

numpy.ndarray

Thus the DataFrame can be thought of as a generalization of a two-dimensional NumPy array, where both the rows and columns have a generalized index for accessing the data.

### DataFrame as specialized dictionary

Similarly, we can also think of a DataFrame as a specialization of a dictionary. Where a dictionary maps a key to a value, a DataFrame maps a column name to a Series of column data. For example, asking for the 'area' attribute returns the Series object containing the areas we saw earlier:

In [32]:
states['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [33]:
type(states['area'])

pandas.core.series.Series

## Constructing DataFrame objects 

A Pandas DataFrame can be constructed in a variety of ways:

### From a single Series object 

A DataFrame is a collection of Series objects, and a single-column DataFrame can be constructed from a single Series:

In [34]:
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [35]:
pd.DataFrame(population, columns=['population'])

,population
California,38332521
Texas,26448193
New York,19651127
Florida,19552860
Illinois,12882135


### From multiple Series 

In [36]:
s1 = pd.Series(['100', '200', 'python', '300.12', '400'])
s2 = pd.Series(['10', '20', 'php', '30.12', '40'])
df = pd.concat([s1, s2], axis = 1)
df

,0,1
0,100,10
1,200,20
2,python,php
3,300.12,30.12
4,400,40


### From a list of dicts 

Any list of dictionaries can be made into a DataFrame. We'll use a simple list comprehension to create some data. Even if some keys in the dictionary are missing, Pandas will fill them in with NaN (i.e., "not a number") values:

In [37]:
df =pd.DataFrame([{'a': 1, 'b': 2},{'b': 3, 'c': 4}], index=["first_dict", "second_dict"])
df

,a,b,c
first_dict,1.0,2,NaN
second_dict,NaN,3,4.0


As every single column must have a consistent dtype and np.NaN is a float, some of the numbers get coerced into floats:

In [38]:
df['a']

first_dict     1.0
second_dict    NaN
Name: a, dtype: float64

In [39]:
df['b']

first_dict     2
second_dict    3
Name: b, dtype: int64

In [40]:
type(np.NaN)

float

In [41]:
df.dtypes

a    float64
b      int64
c    float64
dtype: object

If we wanted to get the rows, pandas would need to coerce the numbers explicitly:

In [42]:
df

,a,b,c
first_dict,1.0,2,NaN
second_dict,NaN,3,4.0


In [43]:
df.loc['first_dict']

a    1.0
b    2.0
c    NaN
Name: first_dict, dtype: float64

### From a two-dimensional NumPy array 

Given a two-dimensional array of data, we can create a DataFrame with any specified column and index names. If omitted, an integer index will be used for each:

In [44]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2013-01-01,0.558989,0.037832,-0.419226,0.562406
2013-01-02,-0.220523,-0.637778,0.388594,-0.226439
2013-01-03,0.074486,0.757664,0.481589,-1.296448
2013-01-04,1.191694,-0.477422,-0.711565,-0.802139
2013-01-05,0.446463,0.228962,-0.241130,0.842474
2013-01-06,0.137598,0.686652,-0.431164,0.338692


## The Pandas Index Object 

We have seen here that both the Series and DataFrame objects contain an explicit index that lets you reference and modify data. This Index object is an interesting structure in itself, and it can be thought of as an immutable array:

In [45]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64')

In [46]:
ind[0]

2

In [47]:
sr = pd.Series(0, index=ind)

In [48]:
sr

2     0
3     0
5     0
7     0
11    0
dtype: int64

Index objects have a name:

In [49]:
ind.names = ['indexx']
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64', name='indexx')

In [50]:
sr = pd.Series(np.zeros_like(ind), index=ind)
sr

indexx
2     0
3     0
5     0
7     0
11    0
dtype: int64

Index objects also have many of the attributes familiar from NumPy arrays:

In [51]:
ind.size, ind.shape, ind.ndim, ind.dtype

(5, (5,), 1, dtype('int64'))

While viewing Indices as immutable list is natural, indices also allow for set-operations:

In [52]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [53]:
indA & indB

Int64Index([3, 5, 7], dtype='int64')

In [54]:
indA ^ indB

Int64Index([1, 2, 9, 11], dtype='int64')

## Data Indexing and Selection

From the numpy lecture, we already know about indexing, slicing, masking, and fancy indexing:

In [55]:
a = np.arange(16).reshape(4,4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [56]:
a[:, [1, 3]][a[:, [1, 3]] % 3 == 0] 
# takes those values of second and fourth column that are divisible by 3 

array([ 3,  9, 15])

Here we'll look at similar means of accessing and modifying values in Pandas Series and DataFrame objects. The corresponding patterns in Pandas are very similar to those of numpy, though there are a few quirks to be aware of.

We'll start with the simple case of the one-dimensional Series object, and then move on to the more complicated two-dimensional DataFrame object.

## Data Selection in Series 

As we saw in the previous section, a Series object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary. If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as dictionary 

Like a dictionary, the Series object provides a mapping from a collection of keys to a collection of values, which means most of the corresponding functions work just as well for them:

In [57]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [58]:
data.__contains__('b')

True

In [59]:
'b' in data

True

In [60]:
np.array_equal(data.keys(), data.index)

True

In [61]:
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [62]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

In [63]:
data['e'] = 1.25

In [64]:
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

### Series as one-dimensional array 

Series builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, slices, masking, and fancy indexing. Examples of these are as follows:

In [65]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [66]:
#slicing by implicit integer index
data[0:2]
# Note that when slicing with an explicit index (i.e., data['a':'c']), the final index is included in the slice, 
# while when slicing with an implicit index (i.e., data[0:2]), the final index is excluded from the slice.

a    0.25
b    0.50
dtype: float64

In [67]:
(data > 0.3) & (data <  0.8)

a    False
b     True
c     True
d    False
e    False
dtype: bool

In [68]:
#masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [69]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

In [70]:
data = pd.Series([0.25, 0.5, 0.75, 1],
                index=[1, 2, 3, 4])
data

1    0.25
2    0.50
3    0.75
4    1.00
dtype: float64

In [71]:
data[1:3]

2    0.50
3    0.75
dtype: float64

**If your Series has an explicit integer index, an indexing operation such as data[1] will use the explicit indices, while a slicing operation like data[1:3] will use the implicit Python-style index.**

In [72]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [73]:
# explicit index when indexing
data[1]

'a'

In [74]:
# implicit indexing when slicing
data[1:3]

3    b
5    c
dtype: object

The **loc** attribute allows indexing and slicing that always references the explicit index:

In [75]:
data.loc[1]

'a'

In [76]:
data.loc[1:3]

1    a
3    b
dtype: object

The **iloc** attribute allows indexing and slicing that always references the implicit Python-style index:

In [77]:
data.iloc[1]

'b'

In [78]:
data.iloc[1:3]

3    b
5    c
dtype: object

## Data Selection in DataFrame 

Recall that a DataFrame acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of Series structures sharing the same index. These analogies can be helpful to keep in mind as we explore data selection within this structure.

In [79]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area': area, 'pop': pop, 'T': 'T'})
data

,area,pop,T
California,423967,38332521,T
Texas,695662,26448193,T
New York,141297,19651127,T
Florida,170312,19552860,T
Illinois,149995,12882135,T


Note that if we index a DataFrame, we index the **column!!**

In [80]:
# Dictionary-style indexing results in a Series
print(type(data["area"]))
data["area"]

<class 'pandas.core.series.Series'>


California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [81]:
# we can also dereference, though it leads to side-effects if that's actually also a method
data.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [82]:
type(data.area)

pandas.core.series.Series

With this picture in mind, many familiar array-like observations can be done on the DataFrame itself. For example, we can transpose the full DataFrame to swap rows and columns:

In [83]:
data.T

,California,Texas,New York,Florida,Illinois
area,423967,695662,141297,170312,149995
pop,38332521,26448193,19651127,19552860,12882135
T,T,T,T,T,T


For array-style indexing, Pandas again uses the loc and iloc indexers mentioned earlier. Using the iloc indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), but the DataFrame index and column labels are maintained in the result:

In [84]:
data.values[:3, :2]

array([[423967, 38332521],
       [695662, 26448193],
       [141297, 19651127]], dtype=object)

In [85]:
data.iloc[:3, :2]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


In [86]:
data

,area,pop,T
California,423967,38332521,T
Texas,695662,26448193,T
New York,141297,19651127,T
Florida,170312,19552860,T
Illinois,149995,12882135,T


In [87]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [88]:
data.loc[:, ['area', 'pop']]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


So, this is how we get a row!

In [89]:
data.loc['California', :]

area      423967
pop     38332521
T              T
Name: California, dtype: object

In [91]:
# adding a new column
data['density'] = data['pop'] / data['area']
# we can combine masking with fancy indexing
data.loc[data.density > 100, ['pop','density']]

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


While indexing refers to columns, slicing refers to rows:

In [92]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [93]:
data['Florida':'Illinois']

,area,pop,T,density
Florida,170312,19552860,T,114.806121
Illinois,149995,12882135,T,85.883763
